Задача 1

    1.1 Условие
На вход доступен следующий фрейм данных:
id	timestamp
1	1562007679
1	1562007710
1	1562007720
1	1562007750
2	1564682430
2	1564682450
2	1564682480
id — уникальный идентификатор пользователя
timestamp — время действия на сайте, совершенное данным пользователем

    1.2 Задание
Для каждого id рассчитайте усредненную длину сессии в рамках суток. Усредненная длина сессии рассчитывается как разница между первым и последним действием, произведенным на сайте.
Подсказка: воспользуйтесь возможностями оконных функций в Spark.


In [2]:
# Импортирование необходимой библиотеки pandas
import pandas as pd

In [3]:
# Создание датафрейма с помощью pandas из словаря:
data1 = {'id': ['1', '1',  '1', '1', '2', '2', '2'],
        'timestamp': ['1562007679', '1562007710', '1562007720', '1562007750'
                      , '1564682430', '1564682450', '1564682480'] 
        }
df1 = pd.DataFrame(data1)
df1 = df1.astype({'id': 'string', 'timestamp' : 'int'})

In [4]:
# Импортирование необходимых библиотек pyspark
from pyspark.sql import SparkSession

In [5]:
# Инициализация Spark-сессии
spark = SparkSession.builder \
    .master("local") \
    .appName("onlineretail") \
    .getOrCreate()

24/08/15 21:36:21 WARN Utils: Your hostname, alto-pc resolves to a loopback address: 127.0.1.1; using 192.168.181.138 instead (on interface eth0)
24/08/15 21:36:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/15 21:36:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/15 21:36:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
# Создание dataframe для pyspark
sparkDF = spark.createDataFrame(df1) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- id: string (nullable = true)
 |-- timestamp: long (nullable = true)



+---+----------+
| id| timestamp|
+---+----------+
|  1|1562007679|
|  1|1562007710|
|  1|1562007720|
|  1|1562007750|
|  2|1564682430|
|  2|1564682450|
|  2|1564682480|
+---+----------+



Для каждого id рассчитайте усредненную длину сессии в рамках суток. 

Усредненная длина сессии рассчитывается как разница между первым и последним действием, произведенным на сайте.

Подсказка: воспользуйтесь возможностями оконных функций в Spark.

In [7]:
# Импорт оконных функций
from pyspark.sql.window import Window
from pyspark.sql.functions import col,min,max 

# Основание для разбиения данных
sesLenAgg  = Window.partitionBy("id")

# Добавление колонки со значением усредненной длины сессии (UDS)
sparkDF.withColumn("UDS", max(col("timestamp")).over(sesLenAgg)-min(col("timestamp")).over(sesLenAgg)) \
  .show()


+---+----------+---+
| id| timestamp|UDS|
+---+----------+---+
|  1|1562007679| 71|
|  1|1562007710| 71|
|  1|1562007720| 71|
|  1|1562007750| 71|
|  2|1564682430| 50|
|  2|1564682450| 50|
|  2|1564682480| 50|
+---+----------+---+



In [8]:
# Остановка spark-сессии
spark.stop()